
The Salary Dataset contains **6704 rows** and **6 columns** containing the following data:

1. **Age**
2. **Gender**
3. **Education Level**
4. **Job Title**
5. **Years of Experience**
6. **Salary**

First we pre-process, clean and model the data to standarsise and structure it.


# Importing Libraries

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
### Suggestions for Improvement

1. **Documentation:**
   - Add detailed descriptions and docstrings to the code.
   - Provide a few example datasets or a link to where they can be obtained.

2. **Testing:**
   - Include unit tests for different components of the project.

3. **Error Handling:**
   - Add error handling for common issues (e.g., missing files, incorrect data formats).

4. **Enhance the GUI:**
   - Improve the GUI for better user experience.

5. **Project Automation:**
   - Add CI/CD pipelines to automate testing and deployment.

6. **Code Quality:**
   - Ensure code adheres to PEP8 standards.
```

**Next steps:**
**a.** Add unit tests for the `app.py` script.
**b.** Enhance the GUI with additional features for better usability.
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error 

In [ ]:
# Importing Data

df = pd.read_csv(r'../dataset/Salary_Data.csv')
# Dropping null values from database
df.dropna(inplace=True)
# Checking unique value counts of Job Titles in the database
# print(df['Job Title'].value_counts())

# Reducing Job titles by omitting titles with less than 25 counts

job_title_count = df['Job Title'].value_counts()
job_title_edited = job_title_count[job_title_count<=25]
job_title_edited.count()

# Omitting titles with less than 25 counts

df['Job Title'] = df['Job Title'].apply(lambda x: 'Others' if x in job_title_edited else x )
print(df['Job Title'].to_list())
# print(df.head())

#Checking unique value count of Education Level

df['Education Level'].value_counts()

# Combining repeating values of education level

df['Education Level'].replace(["Bachelor's Degree","Master's Degree","phD"],["Bachelor's","Master's","PhD"],inplace=True)
df['Education Level'].unique()


In [ ]:
# Creating dummies for Job titles
dummies = pd.get_dummies(df['Job Title'],drop_first=True)
df = pd.concat([df,dummies],axis=1)

# Drop Job Title column
df.drop('Job Title',inplace=True,axis=1)
# Label encoding the categorical variable
le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])
# Mapping Education Level column
education_mapping = {"High School": 0, "Bachelor's": 1, "Master's": 2, "PhD": 3}
df['Education Level'] = df['Education Level'].map(education_mapping)
df.head()



**Predicting Salary**

3 Models will be used to predict the salary

1. Linear Regression
2. Deision Tree
3. Random Forest

In [ ]:
# detecting the outliers in salary column using IQR method
Q1 = df.Salary.quantile(0.25) # First Quartile
Q3 = df.Salary.quantile(0.75) # Third Quartile

# Caltulation Interquartile
IQR = Q3-Q1

# Deetecting outliers lying 1.5x of IQR above and below Q1 and Q3 resp
lower = Q1-1.5*IQR
upper = Q3+1.5*IQR

In [ ]:
df[df.Salary>upper]

**No outliers in Q3**

In [ ]:
df[df.Salary<lower]

**No outliers in Q1**

<h2>Preparing the data for ML analysis by converting categorical job titles into a numerical format<h2>

In [ ]:
# Separating the dataset into features and target

# Dataset conntaining all features(X) from df
X = df.drop('Salary',axis=1)

# Series containing target(Y) variable to be predicted
Y= df['Salary']

In [ ]:
# Splitting data into 25% training and 75% test sets

x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.25,random_state=42)
x_train.shape

In [ ]:
# Create a dictionary for defining models and tuning hyperparameters

model_params = {
    'Linear_Regression':{
        'model':LinearRegression(),
        'params':{
            
        }
    },
    'Decision_Tree':{
        'model':DecisionTreeRegressor(),
        'params':{
            'max_depth':[2,4,6,8,10],
            'random_state':[0,42],
            'min_samples_split':[1,5,10,20]
        }
    },
    'Random_Forest':{
        'model':RandomForestRegressor(),
        'params':{
            'n_estimators':[10,30,20,50,80]
        }
    }
}

In [ ]:
# Hyper parameter tuning through grid search cv
score=[]

for model_name,m in model_params.items():
    clf = GridSearchCV(m['model'],m['params'],cv=5,scoring='neg_mean_squared_error')
    clf.fit(x_train,y_train)
    
    score.append({
        'Model':model_name,
        'Params':clf.best_params_,
        'MSE(-ve)':clf.best_score_
    })
pd.DataFrame(score)    

In [ ]:
# Order of the best models 

s = pd.DataFrame(score)
sort = s.sort_values(by = 'MSE(-ve)',ascending=False)
sort

In [ ]:
# Random Forest model

rfr = RandomForestRegressor(n_estimators=20)
rfr.fit(x_train,y_train)
print(x_train['Education Level'].value_counts())

In [ ]:
rfr.score(x_test,y_test)
x_test.head()
y_test.head()
        
# y_test.head(1)

In [ ]:
y_pred_rfr = rfr.predict(x_test)

print("Mean Squared Error :",mean_squared_error(y_test,y_pred_rfr))
print("Mean Absolute Error :",mean_absolute_error(y_test,y_pred_rfr))
print("Root Mean Squared Error :",mean_squared_error(y_test,y_pred_rfr,squared=False))

In [ ]:
# Decision Tree model

dtr = DecisionTreeRegressor(max_depth=10,min_samples_split=2,random_state=0)
dtr.fit(x_train,y_train)

In [ ]:
dtr.score(x_test,y_test)

In [ ]:
y_pred_dtr = dtr.predict(x_test)

print("Mean Squared Error :",mean_squared_error(y_test,y_pred_dtr))
print("Mean Absolute Error :",mean_absolute_error(y_test,y_pred_dtr))
print("Root Mean Squared Error :",mean_squared_error(y_test,y_pred_dtr,squared=False))

In [ ]:
# Linear regression model

lr = LinearRegression()
lr.fit(x_train,y_train)

In [ ]:
lr.score(x_test,y_test)

In [ ]:
y_pred_lr = dtr.predict(x_test)
plt.figure(figsize=(10, 6))
plt.scatter(y_pred_lr, y_test, color='blue', marker='o', alpha=0.7)
plt.xlabel('predicted', fontsize=12)
plt.ylabel('original', fontsize=12)
plt.title('pridicted vs original')
plt.tight_layout()
plt.show()


print("Mean Squared Error :",mean_squared_error(y_test,y_pred_lr))
print("Mean Absolute Error :",mean_absolute_error(y_test,y_pred_lr))
print("Root Mean Squared Error :",mean_squared_error(y_test,y_pred_lr,squared=False))

In [ ]:
# Access the feature importances of Random Forest Regressor
feature_importances = rfr.feature_importances_

# Assuming you have a list of feature names that corresponds to the feature importances
feature_names = list(x_train.columns)

# Sort feature importances in descending order
sorted_indices = np.argsort(feature_importances)[::-1]
sorted_feature_importances = [feature_importances[i] for i in sorted_indices]
sorted_feature_names = [feature_names[i] for i in sorted_indices]

# Create a bar chart
plt.figure(figsize=(12, 8))
plt.barh(sorted_feature_names[:10], sorted_feature_importances[:10])
plt.xlabel('Feature Importance')
plt.title('Top 10 Feature Importance in Predicting Salary')
plt.gca().invert_yaxis()  # Invert the y-axis for better visualization
plt.show()


**A bar chart depicting the importance of different features in predicting salary.**

<h2>Conclusion<h2>

**1. The Random Forest model achieved the highest R-squared score (0.971) and the lowest MSE, MAE, and RMSE values, indicating the best predictive performance among the three models.**

**2. The Decision Tree model performed well with an R-squared score of 0.941 but had higher errors compared to the Random Forest.**

**3. The Linear Regression model had the lowest R-squared score (0.833) and the highest errors, suggesting it may not capture the underlying patterns in the data as effectively as the ensemble models.**

In conclusion, the Random Forest model appears to be the most suitable for predicting salaries in this dataset, as it offers the highest predictive accuracy and the lowest error metrics. Further optimization and fine-tuning of the Random Forest model could potentially lead to even better results.

# Save the model
save the model in pkl file

In [ ]:
#! Saving the model as a pickle file !!!
'''
import pickle

# save random forest model as a pickle file 
model_pkl_file = "salary predictor(rfr).pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(rfr, file)

# save linear regression model as a pickle file 
model_pkl_file = "salary predictor(lr).pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(lr, file)
    
# save Decsion tree model as a pickle file 
    
model_pkl_file = "salary predictor(dtr).pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(dtr, file)
model_pkl_file = "df.pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(df, file)
'''